Importando Libs

In [1]:
import pyodbc
import pandas as pd
import warnings
from sqlalchemy import create_engine 
import urllib
import os

warnings.filterwarnings('ignore', category=UserWarning)

In [3]:
senha = os.getenv('senha_db')
print(os.getenv('senha_db'))

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"\
    "SERVER=den1.mssql7.gear.host;"\
    "DATABASE=vendasbicyclon;"\
    "UID=vendasbicyclon;"\
    f"PWD={senha};"\
    "MARS_Connection=Yes"
)

senha@clarify


In [ ]:
def exe_query(query):
    return pd.read_sql(query, conn)

lista_resposta = []
def exe_varias_querys(*args):
    for consulta in args:
        lista_resposta.append(pd.read_sql(consulta,conn))
    return lista_resposta

In [ ]:
# from pyspark.sql import SparkSession 

# SparkSession = SparkSession.builder \
# .master("local[2]") \
# .appName("Teste") \
# .getOrCreate()

In [ ]:

# params = urllib.parse.quote_plus(
#     "DRIVER={ODBC Driver 17 for SQL Server};"\
#     "SERVER=den1.mssql7.gear.host;"\
#     "DATABASE=vendasbicyclon;"\
#     "UID=vendasbicyclon;"\
#     "PWD=senha@clarify;"\
#     "MARS_Connection=Yes")

# engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')

Produtos, total de pedidos e qtd vendida

In [ ]:
q = """ 
SELECT 
    produtos.nomeProduto,
    count(itensPedido.idItem) as total_pedidos,
    sum(itensPedido.quantidade) as qtd_vendido

    from produtos join itensPedido
        on produtos.idProduto = itensPedido.fkProduto
        group by produtos.nomeProduto
"""

exe_query(q)

,nomeProduto,total_pedidos,qtd_vendido
0,Caloi Elite Carbon,26,45
1,Caloi Urbam,35,68
2,Capacete Giro Register,35,62
3,Cassete Shimano 11v,39,89
4,Freio a Disco Hidráulico Shimano,36,72
5,Grupo Shimano Deore,37,80
6,Luva Ciclismo Gel,35,62
7,Luz Traseira Recarregável,28,54
8,Oggi Lite Tour,33,72
9,Sense Impact Pro,25,53


In [ ]:
q_vw = """
if OBJECT_ID('vwFatoVendas', 'V') is not null
    drop view vwFatoVendas

exec('
    create view vwFatoVendas as
    select 
        pedidos.idPedido,
        pedidos.dataPedido,
        clientes.nomeCliente,
        vendedores.nomeVendedor,
        regionais.nomeRegional,
        produtos.nomeProduto,
        linhasProdutos.nomeLinhaProduto,
        itensPedido.quantidade,
        itensPedido.precoUnitario,
        (itensPedido, quantidade * (itensPedido.precoUnitario - itensPedido.desconto)) as valor_total
        
        from pedidos
        join clientes on pedidos.fkCliente = clientes.idPedido
        join vendedores on pedidos.fkVendedor = vendedores.idVendedor
        join regionais on vendedores.fkRegional = regionais.idRegional
        join itensPedido on itensPedido.fkPedido = pedidos.idPedido
        join produtos on itensPedido.fkProduto = produtos.idProduto
        join linhasProdutos on produtos.fkLinhaProduto = linhasProdutos.idLinhaProduto
')
"""

cursor = conn.cursor()
cursor.execute(q_vw)
conn.commit()

df_fatovendas = pd.read_sql('select * from vwFatoVendas')

df_fatovendas.columns

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Incorrect syntax near ','. (102) (SQLExecDirectW)")